In [ ]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible2.sliced import sliced_from_samples
from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible2.splitter import SubsampleSplitter

from reversible2.view_as import ViewAs
from reversible2.invert import invert

from reversible2.affine import AdditiveBlock
from reversible2.plot import display_text, display_close
from reversible2.bhno import load_file, create_inputs

In [ ]:
# 2 gaussians different stds and means.. or uniform dims..
# train, show what would happen for different visualizations


In [ ]:
rng = RandomState(201904113)#2 ganz gut # 13 sehr gut)

X = rng.randn(5,2) * np.array([1,0])[None] + np.array([-1,0])[None]


plt.figure(figsize=(5,5))
plt.scatter(X[:,0], X[:,1])
plt.scatter([-1],[0], color='black')
plt.xlim(-2.5,5.5)
plt.ylim(-4,4)

In [ ]:
train_inputs = np_to_var(X, dtype=np.float32)

In [ ]:
cuda = False

In [ ]:
from reversible2.distribution import TwoClassDist

from reversible2.blocks import dense_add_block, conv_add_block_3x3
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter


set_random_seeds(2019011641, cuda)
feature_model_a = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)
feature_model_b = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)
if cuda:
    feature_model_a.cuda()
    feature_model_b.cuda()
from reversible2.ot_exact import ot_euclidean_loss_for_samples
class_dist = TwoClassDist(2,0)
if cuda:
    class_dist.cuda()

optim_model_a = th.optim.Adam(feature_model_a.parameters())
optim_model_b = th.optim.Adam(feature_model_b.parameters())
optim_dist = th.optim.Adam(class_dist.parameters(), lr=1e-3)

In [ ]:
from reversible.gaussian import get_gauss_samples

In [ ]:
n_epochs = 2001
for i_epoch in range(n_epochs):
    outs = feature_model_a(train_inputs)

    loss = th.sum(outs[:,0] ** 2) + th.sum(outs[:,1] ** 2) * 100
    optim_model_a.zero_grad()
    loss.backward()
    optim_model_a.step()
    if i_epoch % (n_epochs // 20) == 0:
        inverted = invert(feature_model_a, get_gauss_samples(100, th.zeros(2), np_to_var([1,0], dtype=np.float32)))
        fig = plt.figure(figsize=(3,3))
        for i_class in range(len(train_inputs)):
            outs = feature_model_a(train_inputs)
            plt.scatter(var_to_np(inverted)[:,0], var_to_np(inverted)[:,1])
        plt.title("Samples 1d\nLoss: {:.2E}".format(loss.item()) )
        display_close(fig)
        inverted = invert(feature_model_a, get_gauss_samples(100, th.zeros(2), np_to_var([1,1], dtype=np.float32)))
        fig = plt.figure(figsize=(3,3))
        for i_class in range(len(train_inputs)):
            outs = feature_model_a(train_inputs)
            plt.scatter(var_to_np(inverted)[:,0], var_to_np(inverted)[:,1])
        plt.title("Samples 2d\nLoss: {:.2E}".format(loss.item()) )
        display_close(fig)
        fig = plt.figure(figsize=(3,3))
        for i_class in range(len(train_inputs)):
            outs = feature_model_a(train_inputs)

            plt.scatter(var_to_np(outs)[:,0], var_to_np(outs)[:,1])
        plt.title("Outs\nLoss: {:.2E}".format(loss.item()) )
        display_close(fig)

In [ ]:
n_epochs = 2001
for i_epoch in range(n_epochs):
    outs = feature_model_a(train_inputs)

    loss = th.sum(outs[:,0] ** 2) + th.sum(outs[:,1] ** 2) * 100
    optim_model_a.zero_grad()
    loss.backward()
    optim_model_a.step()
    if i_epoch % (n_epochs // 20) == 0:
        inverted = invert(feature_model_a, get_gauss_samples(100, th.zeros(2), np_to_var([1,0], dtype=np.float32)))
        fig = plt.figure(figsize=(3,3))
        for i_class in range(len(train_inputs)):
            outs = feature_model_a(train_inputs)
            plt.scatter(var_to_np(inverted)[:,0], var_to_np(inverted)[:,1])
        plt.title("Samples 1d\nLoss: {:.2E}".format(loss.item()) )
        display_close(fig)
        inverted = invert(feature_model_a, get_gauss_samples(100, th.zeros(2), np_to_var([1,1], dtype=np.float32)))
        fig = plt.figure(figsize=(3,3))
        for i_class in range(len(train_inputs)):
            outs = feature_model_a(train_inputs)
            plt.scatter(var_to_np(inverted)[:,0], var_to_np(inverted)[:,1])
        plt.title("Samples 2d\nLoss: {:.2E}".format(loss.item()) )
        display_close(fig)
        fig = plt.figure(figsize=(3,3))
        for i_class in range(len(train_inputs)):
            outs = feature_model_a(train_inputs)

            plt.scatter(var_to_np(outs)[:,0], var_to_np(outs)[:,1])
        plt.title("Outs\nLoss: {:.2E}".format(loss.item()) )
        display_close(fig)

In [ ]:
outs = feature_model_a(train_inputs)

fig = plt.figure(figsize=(6,6))
plt.scatter(var_to_np(train_inputs[:,0]), var_to_np(train_inputs[:,1]))
plt.scatter(var_to_np(outs[:,0]), var_to_np(outs[:,1]))
for a,b in zip(var_to_np(outs), var_to_np(train_inputs)):
    plt.annotate("", xy=a, xytext=b,
                 arrowprops=dict(arrowstyle="->",),
                    )

display_close(fig)

In [ ]:
rng = RandomState(201904114)
outs = feature_model_a(train_inputs)
other_X = rng.randn(100,2) * np.array([1,0])[None] + np.array([-1,0])[None]
other_ins = np_to_var(other_X, dtype=np.float32)
other_outs = feature_model_a(other_ins)

fig = plt.figure(figsize=(6,6))
plt.scatter(var_to_np(other_ins[:,0]), var_to_np(other_ins[:,1]))
plt.scatter(var_to_np(other_outs[:,0]), var_to_np(other_outs[:,1]))
plt.scatter(var_to_np(train_inputs[:,0]), var_to_np(train_inputs[:,1]))
plt.scatter(var_to_np(outs[:,0]), var_to_np(outs[:,1]), s=30)
for a,b in zip(var_to_np(other_ins), var_to_np(other_outs)):
    plt.annotate("", xy=b, xytext=a,
                 arrowprops=dict(arrowstyle="->",lw=0.2),
                    )
for a,b in zip(var_to_np(train_inputs), var_to_np(outs)):
    plt.annotate("", xy=b, xytext=a,
                 arrowprops=dict(arrowstyle="->",lw=2),
                    )
plt.legend(("Inputs", "Outputs"))
display_close(fig)

In [ ]:
rng = RandomState(201904114)

other_X = rng.randn(100,2) * np.array([1,0])[None] + np.array([-1,0])[None]
other_ins = np_to_var(other_X, dtype=np.float32)
other_outs = feature_model_a(other_ins)

fig = plt.figure(figsize=(6,6))
plt.scatter(var_to_np(other_ins[:,0]), var_to_np(other_ins[:,1]))
plt.scatter(var_to_np(other_outs[:,0]), var_to_np(other_outs[:,1]))
xmin= -0.1
xmax = 0.2
plt.xlim(xmin,xmax)
for a,b in zip(var_to_np(other_ins), var_to_np(other_outs)):
    plt.annotate("", xy=np.clip(b,xmin,xmax), xytext=np.clip(a,xmin,xmax),
                 arrowprops=dict(arrowstyle="->",lw=0.2),
                )
for a,b in zip(var_to_np(train_inputs), var_to_np(outs)):
    plt.annotate("", xy=np.clip(b,xmin,xmax), xytext=np.clip(a,xmin,xmax),
                 arrowprops=dict(arrowstyle="->",lw=1),
                )
plt.scatter(var_to_np(train_inputs[:,0]), var_to_np(train_inputs[:,1]), s=30)
plt.scatter(var_to_np(outs[:,0]), var_to_np(outs[:,1]), s=30)
plt.legend(("Inputs", "Outputs", "TIns", "TOuts"))
display_close(fig)

In [ ]:
Capacity: gradient 

In [ ]:
rng = RandomState(201904114)
samples = get_gauss_samples(100, th.zeros(2), np_to_var([1,1], dtype=np.float32))
inverted = invert(feature_model_a, samples)
fig = plt.figure(figsize=(6,6))
plt.scatter(var_to_np(inverted[:,0]), var_to_np(inverted[:,1]))
plt.scatter(var_to_np(samples[:,0]), var_to_np(samples[:,1]))
plt.scatter(var_to_np(train_inputs[:,0]), var_to_np(train_inputs[:,1]), s=30)
plt.scatter(var_to_np(outs[:,0]), var_to_np(outs[:,1]), s=30)
xmin= -10
xmax = 10
plt.xlim(xmin,xmax)
plt.ylim(xmin,xmax)
for a,b in zip(var_to_np(samples), var_to_np(inverted)):
    plt.annotate("", xy=np.clip(b,xmin,xmax), xytext=np.clip(a,xmin,xmax),
                 arrowprops=dict(arrowstyle="->",lw=0.2),
                )
for a,b in zip(var_to_np(outs), var_to_np(train_inputs)):
    plt.annotate("", xy=np.clip(b,xmin,xmax), xytext=np.clip(a,xmin,xmax),
                 arrowprops=dict(arrowstyle="->",lw=1),
                )
plt.legend(("Inputs", "Outputs", "TIns", "TOuts"))
display_close(fig)

In [ ]:
rng = RandomState(201904114)
samples = get_gauss_samples(100, th.zeros(2), np_to_var([1,1], dtype=np.float32))
inverted = invert(feature_model_a, samples)
fig = plt.figure(figsize=(6,6))
plt.scatter(var_to_np(inverted[:,0]), var_to_np(inverted[:,1]))
plt.scatter(var_to_np(samples[:,0]), var_to_np(samples[:,1]))
plt.scatter(var_to_np(train_inputs[:,0]), var_to_np(train_inputs[:,1]), s=30)
plt.scatter(var_to_np(outs[:,0]), var_to_np(outs[:,1]), s=30)
xmin= -0.5
xmax = 0.5
plt.xlim(xmin,xmax)
plt.ylim(xmin,xmax)
for a,b in zip(var_to_np(samples), var_to_np(inverted)):
    plt.annotate("", xy=np.clip(b,xmin,xmax), xytext=np.clip(a,xmin,xmax),
                 arrowprops=dict(arrowstyle="->",lw=0.2),
                )
for a,b in zip(var_to_np(outs), var_to_np(train_inputs)):
    plt.annotate("", xy=np.clip(b,xmin,xmax), xytext=np.clip(a,xmin,xmax),
                 arrowprops=dict(arrowstyle="->",lw=1),
                )
plt.legend(("Inputs", "Outputs", "TIns", "TOuts"))
display_close(fig)

In [ ]:
np.std((X[:,0]))

In [ ]:
th.std(outs[:,0])

In [ ]:
th.sum(th.std(outs, dim=0))